In [ ]:
import numpy as np
import os
import pathlib
import xarray as xr

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
RDPS_DIR = DATA_DIR / '2021-01-05-datamart-sample/rdps/'

In [ ]:
rdps_path = pathlib.Path(RDPS_DIR)
temperature_files = [
    *[rdps_path / '2020112600' / 'CMC_reg_TMP_TGL_2_ps10km_2020112600_P{:03}.grib2'.format(t) for t in range(12, 37, 3)],
]

In [ ]:
temperature_files

In [ ]:
tt = xr.open_mfdataset(temperature_files, engine='cfgrib', concat_dim='step', combine='nested', parallel=True)

In [ ]:
tt = xr.open_dataset(temperature_files[0], engine='cfgrib')

In [ ]:
tt

In [ ]:
tt.latitude

In [ ]:
tt.longitude

In [ ]:
point = np.array([45, 290])

In [ ]:
lat_lon = np.stack([tt.latitude, tt.longitude], axis=-1)

In [ ]:
diff = np.square(lat_lon - point)

In [ ]:
dists = np.sqrt(np.mean(diff, axis=-1))

In [ ]:
np.argmin(dists)

In [ ]:
np.unravel_index(337378, dists.shape)

In [ ]:
dists[np.unravel_index(337378, dists.shape)]

In [ ]:
amin = np.argmin(dists)
found_coords = np.unravel_index(amin, dists.shape)

In [ ]:
lat_lon[found_coords]

In [ ]:
found_coords

In [ ]:
found_coords + np.array([-1, 0])

In [ ]:
dists[found_coords + np.array([-1, 0])]

In [ ]:
top = (found_coords[0] + 1, found_coords[1])
bottom = (found_coords[0] - 1, found_coords[1])

print(top)
print(bottom)

if dists[top] < dists[bottom]:
    delta_lat = -1
else:
    delta_lat = 1
    
    
left = (found_coords[0], found_coords[1] - 1)
right = (found_coords[0], found_coords[1] + 1)

delta_lon = -1 if dists[left] < dists[right] else 1

In [ ]:
delta_lat, delta_lon

In [ ]:
bounds = [
    (found_coords[0], found_coords[1]),
    (found_coords[0], found_coords[1] + delta_lon),
    (found_coords[0] + delta_lat, found_coords[1]),
    (found_coords[0] + delta_lat, found_coords[1] + delta_lon),
]

In [ ]:
bounds

In [ ]:
dists[bounds[0]]

In [ ]:
distances = [dists[c] for c in bounds]

In [ ]:
distances

In [ ]:
vec_x = bounds[1] - bounds[0]

In [ ]:
closest_point = lat_lon[found_coords]

In [ ]:
closest_point

In [ ]:
point - closest_point

In [ ]:
point

In [ ]:
v1 = lat_lon[bounds[1]] - lat_lon[bounds[0]]

In [ ]:
v1

In [ ]:
v2 = lat_lon[bounds[2]] - lat_lon[bounds[0]]

In [ ]:
v2

In [ ]:
v1

In [ ]:
vx = point - lat_lon[bounds[0]]

In [ ]:
vx

In [ ]:
# scalar projection vx on v1

In [ ]:
np.dot(vx, v1 / np.linalg.norm(v1))

In [ ]:
# scalar projection vx on v2
np.dot(vx, v2 / np.linalg.norm(v2))

In [ ]:
 v2 / np.linalg.norm(v2)

In [ ]:
v2

In [ ]:
vx

In [ ]:
def bilinear_interpolation(x, y, x1, y1, x2, y2, Q):
    left = np.array([[x2 - x, x - x1]])
    right = np.array([[y2 - y],
                     [y - y1]])
    return (1. / (x2 - x1) * (y2 - y1)) * np.dot(left, np.dot(Q, right))

In [ ]:
np.dot(Q, right)

In [ ]:
np.array(bounds)

In [ ]:
qs = [tt.isel(step=0).to_array().squeeze()[x].compute().data for x in bounds]

In [ ]:
qs

In [ ]:
Q = np.array([[qs[0], qs[1]],[qs[2], qs[3]]])
Q

In [ ]:
bilinear_interpolation(point[0], point[1], v1[0], v1[1], v2[0], v2[1], Q)

#    interpolate lat, lon then value?

In [ ]:
tt

In [ ]:
tt.latitude.interp()

# weigh by distance

In [ ]:
total_dist = sum(dists[x] for x in bounds)

In [ ]:
total_dist

In [ ]:
w = [(total_dist - dists[x]) / total_dist for x in bounds]

In [ ]:
w

In [ ]:
ws = [(total_dist - dists[x]) / total_dist for x in bounds]
ws = [sum(ws[:i]) for i in range(len(ws))]

In [ ]:
ws

In [ ]:
dists

In [ ]:
ws = [total_dist - dists[x] for x in bounds]

In [ ]:
total_ws = sum(ws)

In [ ]:
tt_array = tt.isel(step=0).compute()

In [ ]:
tt_array

In [ ]:
value = sum([(ws[i] * tt_array.t2m[bounds[i]]) / total_ws for i in range(len(bounds))])

In [ ]:
value.data

In [ ]:
tt_array.t2m[bounds[2]]